In [23]:
import pandas as pd
import os, glob

In [24]:
path="/Users/haoranzhang/CS7280/Project3/raw_data"

In [25]:
unit_data

In [26]:
unit_data.head()

,Unnamed: 0,Unnamed: 0.1,Altitude,Latitude,Longitude,RptTimestamp,Unnamed: 6,time,Unnamed: 0.1.1,ambient_air_temperature_degc,...,fleet,hvbatt_max_cell_temp_c,hvbatt_min_cell_temp_c,m_vs,odometer_distance_m,unitID,mil_V,battery_regen_lifetime_wh,hvac_cycle_wh,vehicle_speed_kph
0,0,0,6,25.598759,-80.348663,1643822280,60,2/3/22 1:18,NaN,33,...,9_FL,NaN,NaN,22,55874760,34,3877,8316467,NaN,34.1719
1,1,1,7,25.598858,-80.344643,1643822340,60,2/3/22 1:19,NaN,32,...,9_FL,NaN,NaN,43,55875530,34,3799,8316547,NaN,37.0508
2,2,2,9,25.600439,-80.340057,1643822400,60,2/3/22 1:20,NaN,31,...,9_FL,NaN,NaN,19,55875935,34,3815,8316665,NaN,25.0273
3,3,3,18,25.604402,-80.339905,1643822460,60,2/3/22 1:21,NaN,32,...,9_FL,NaN,NaN,18,55876790,34,3819,8316765,NaN,17.3555
4,4,4,19,25.602583,-80.337822,1643822520,60,2/3/22 1:22,NaN,31,...,9_FL,NaN,NaN,11,55876785,34,3845,8316836,NaN,0.0000


In [30]:
#pick columns we need
cols_picked=['unitID','m_vs','RptTimestamp','Altitude','Latitude','battery_state_of_charge_pct','ambient_air_temperature_degc','battery_available_energy_wh','odometer_distance_m','hvbatt_max_cell_temp_c','hvbatt_min_cell_temp_c']

In [31]:
data_picked=unit_data[cols_picked]

In [32]:
#sort by timestamp
data_picked=data_picked.sort_values(by="RptTimestamp",ascending=True)

In [33]:
data_picked.head()

,unitID,m_vs,RptTimestamp,Altitude,Latitude,battery_state_of_charge_pct,ambient_air_temperature_degc,battery_available_energy_wh,odometer_distance_m,hvbatt_max_cell_temp_c,hvbatt_min_cell_temp_c
148183,34,2,1606752480,0,25.601730,92.5781,35,56724,21647610,29.0,25.0
148184,34,30,1606759140,7,25.620894,85.9375,38,52655,21651095,31.0,28.0
148185,34,28,1606759200,7,25.620894,85.9375,39,52655,21651095,32.0,28.0
148186,34,24,1606759260,9,25.616123,85.9375,36,52655,21651245,32.0,28.0
148187,34,53,1606759320,10,25.613514,85.9375,37,52655,21652015,32.0,28.0


In [12]:
data_picked['hvbatt_cell_temp_c_difference']=data_picked['hvbatt_max_cell_temp_c']-data_picked['hvbatt_min_cell_temp_c']

<ipython-input-12-9b9896c6aef5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_picked['hvbatt_cell_temp_c_difference']=data_picked['hvbatt_max_cell_temp_c']-data_picked['hvbatt_min_cell_temp_c']


In [34]:
#get temp difference
data_picked = data_picked.assign(hvbatt_cell_temp_c_difference = data_picked['hvbatt_max_cell_temp_c']-data_picked['hvbatt_min_cell_temp_c'])

In [36]:
#fill NaN with 0
data_picked['hvbatt_cell_temp_c_difference']=data_picked['hvbatt_cell_temp_c_difference'].fillna(0)

In [37]:
#drop max & min temp
data_picked=data_picked.drop(columns=['hvbatt_max_cell_temp_c','hvbatt_min_cell_temp_c'])

In [49]:
#get time_gap
data_picked['time_gap']=data_picked['RptTimestamp'].shift(-1)-data_picked['RptTimestamp']

In [50]:
#move time_gap to the column 3. 
data_picked.insert(3,"time_gap",data_picked.pop("time_gap"))

In [54]:
#re-index
data_picked=data_picked.reset_index(drop=True)

In [55]:
data_picked.head()

,unitID,m_vs,RptTimestamp,time_gap,Altitude,Latitude,battery_state_of_charge_pct,ambient_air_temperature_degc,battery_available_energy_wh,odometer_distance_m,hvbatt_cell_temp_c_difference
0,34,2,1606752480,6660.0,0,25.601730,92.5781,35,56724,21647610,4.0
1,34,30,1606759140,60.0,7,25.620894,85.9375,38,52655,21651095,3.0
2,34,28,1606759200,60.0,7,25.620894,85.9375,39,52655,21651095,4.0
3,34,24,1606759260,60.0,9,25.616123,85.9375,36,52655,21651245,4.0
4,34,53,1606759320,60.0,10,25.613514,85.9375,37,52655,21652015,4.0


In [57]:
data_picked.to_csv('./cleaned/unit_'+str(34)+'.csv')

In [ ]:
def preprocess(unitID): 
    #read_in csv
    unit_data=pd.read_csv("data_unit_"+str(unitID)+".csv")
    
    #pick columns we need
    cols_picked=['unitID','m_vs','RptTimestamp','Altitude','Latitude','battery_state_of_charge_pct','ambient_air_temperature_degc','battery_available_energy_wh','odometer_distance_m','hvbatt_max_cell_temp_c','hvbatt_min_cell_temp_c']
    data_picked=unit_data[cols_picked]
    
    #sort by timestamp
    data_picked=data_picked.sort_values(by="RptTimestamp",ascending=True)
    
    #get temp difference
    data_picked = data_picked.assign(hvbatt_cell_temp_c_difference = data_picked['hvbatt_max_cell_temp_c']-data_picked['hvbatt_min_cell_temp_c'])
    
    #fill NaN with 0
    data_picked['hvbatt_cell_temp_c_difference']=data_picked['hvbatt_cell_temp_c_difference'].fillna(0)
    
    #drop max & min temp
    data_picked=data_picked.drop(columns=['hvbatt_max_cell_temp_c','hvbatt_min_cell_temp_c'])
    
    #get time_gap
    data_picked['time_gap']=data_picked['RptTimestamp'].shift(-1)-data_picked['RptTimestamp']
    
    #move time_gap to the column 3. 
    data_picked.insert(3,"time_gap",data_picked.pop("time_gap"))
    
    #re-index
    data_picked=data_picked.reset_index(drop=True)
    
    #output as another csv file
    data_picked.to_csv('./cleaned/unit_'+str(unitID)+'.csv')